## Optimization of judge using gold standard labels

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dspy_judge.llm_caller.utils import load_secrets
from dspy_judge.data_loader.dataset_loader import CustomerSupportDatasetLoader
from dspy_judge.processor.parallel_processor import ParallelProcessor
from dspy_judge.prompts.dspy_signatures import SupportTranscriptJudge
from dspy_judge.processor.utils import convert_dataset_to_dspy_examples, extract_llm_response_fields_dspy
from dspy_judge.processor.parallel_processor import ParallelProcessor
from dspy_judge.metrics import match_judge_metric
from dspy_judge.plotting import plot_judge_results
import numpy as np
from sklearn.metrics import cohen_kappa_score
import dspy

/Users/rmartinshort/Documents/DS_projects/dspy_judge/judgemodel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
secrets = load_secrets()

In [4]:
data_loader = CustomerSupportDatasetLoader()

## Set up judge

In [5]:
judge_model = dspy.LM(
    "gemini/gemini-1.5-flash",
    api_key=secrets["GEMINI_API_KEY"],
    cache=False,
    temperature=0
)
dspy.configure(lm=judge_model,track_usage=True,adapter=dspy.JSONAdapter())
generate_judge_reasoning = dspy.ChainOfThought(SupportTranscriptJudge)

## Load the gold standard judge dataset (AKA SME labels)

In [6]:
dspy_gold_standard_judge_results = data_loader.load_local_dataset("datasets/gold_standard_judge_result")

2025-08-23 15:35:51 - dspy_judge.data_loader.dataset_loader - INFO - Local dataset loaded from datasets/gold_standard_judge_result. Size: 160


In [7]:
judge_dataset_examples = convert_dataset_to_dspy_examples(
    dspy_gold_standard_judge_results,
    field_mapping = {"transcript":"output_transcript","satisfied":"satisfied"},
    input_field="transcript"
)

2025-08-23 15:35:53 - dspy_judge.processor.utils - INFO - Processed 160 training examples


## Check that the metric works

In [34]:
evaluator = dspy.Evaluate(
    metric=match_judge_metric,
    devset=judge_dataset_examples,
    display_table=True,
    display_progress=True,
    num_threads=24,
)
original_score = evaluator(generate_judge_reasoning)

15:44:54 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:44:54 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:44:54 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:44:54 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:44:54 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


...

2025-08-23 15:44:58 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 102.00 / 159 (64.2%):  99%|█████████████████████████████████████████████████████████████████████████████████████████████████▊ | 158/160 [00:03<00:00, 34.53it/s]

15:44:58 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


2025-08-23 15:44:58 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 102.00 / 160 (63.8%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:03<00:00, 44.07it/s]

2025/08/23 15:44:58 INFO dspy.evaluate.evaluate: Average Metric: 102 / 160 (63.8%)


,transcript,example_satisfied,_id,reasoning,pred_satisfied,match_judge_metric
0,Company: American Airlines Transcript so far: Customer: I need thi...,True,example_0,"Agent is polite and helpful, offering alternative solutions.",True,✔️ [1]
1,Company: American Airlines Transcript so far: Customer: traveling ...,True,example_1,Agent's response is polite but unhelpful; doesn't offer any soluti...,False,
2,Company: American Airlines Transcript so far: Customer: Looks like...,False,example_2,"The response is polite and acknowledges the customer's concern, bu...",False,✔️ [1]
3,Company: American Airlines Transcript so far: Customer: I am bring...,True,example_3,"The response is polite, helpful, and provides all necessary inform...",True,✔️ [1]
4,Company: American Airlines Transcript so far: Customer: I printed ...,True,example_4,Agent was polite and answered the question clearly and concisely.,True,✔️ [1]
...,...,...,...,...,...,...
155,Company: American Airlines Conversation Transcript so far: Custome...,True,example_155,"The agent is polite and helpful, redirecting the customer appropri...",True,✔️ [1]
156,"Company: American Airlines Transcript so far: Customer: In email, ...",True,example_156,"Good response, polite and acknowledges the problem.",True,✔️ [1]
157,Company: American Airlines Transcript so far: Customer: I am looki...,False,example_157,"The agent is polite and helpful, offering to find the cheapest tic...",True,
158,Company: Delta Air Lines Transcript so far: Customer: I fly from B...,False,example_158,"The response is polite and helpful, offering a solution and reassu...",True,


In [35]:
original_score

63.75

## Check that we can get the same result using the parallel processor

Why do we do this? We need to confirm that when we run the judge on the generator development dataset, we can reproduce the same behavior that we saw in judge development

In [13]:
dspy_judge_config = {
  "model_name":"gemini/gemini-1.5-flash",
  "api_key":secrets["GEMINI_API_KEY"],
  "temperature": 0
}

dspy_judge_processor = ParallelProcessor()

dspy_judge_results = dspy_judge_processor.process_dataset_with_dspy(
  dspy_gold_standard_judge_results.select_columns(
    ["conversation_id","output_transcript"]
  ),
  input_field="output_transcript",
  dspy_module=generate_judge_reasoning,
  dspy_config=dspy_judge_config
)

2025-08-23 15:38:02 - dspy_judge.processor.parallel_processor - INFO - Initialized ParallelProcessor with max_workers=4
2025-08-23 15:38:02 - dspy_judge.processor.parallel_processor - INFO - Processing 160 examples with 4 workers using DSPy...


Processing with DSPy:   0%|                                                                                                                             | 0/160 [00:00<?, ?it/s]15:38:05 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
15:38:05 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
15:38:05 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
15:38:05 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:38:05 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:05 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:05 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:05 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:38:06 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
15:38:06 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
15:38:06 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
15:38:06 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
15:38:06 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
15:38:06 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
15:38:06 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
Processing with DSPy:   1%|▋                                                                                                                    | 1/160 [00:03<09:23,  3.54s/it]15:38:06 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:38:06 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:06 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:06 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:06 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:06 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:06 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:06 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:06 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


...

2025-08-23 15:38:24 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:24 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini
2025-08-23 15:38:24 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2025-08-23 15:38:24 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


15:38:24 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
Processing with DSPy:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 159/160 [00:22<00:00,  9.03it/s]

2025-08-23 15:38:24 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


15:38:25 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler
Processing with DSPy: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:22<00:00,  7.12it/s]

2025-08-23 15:38:25 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


In [14]:
dspy_judge_results = dspy_judge_results.map(
    extract_llm_response_fields_dspy
)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:00<00:00, 20541.43 examples/s]


In [32]:
# this results
baseline_result = dspy_judge_results.to_pandas()[["conversation_id","satisfied"]].rename(
    columns={
        "satisfied":"prediction"
    }
)
# gold standard result
gold_standard_result = dspy_gold_standard_judge_results.to_pandas()[["conversation_id","satisfied"]].rename(
    columns={
        "satisfied":"gold_standard"
    }
)
full_result = baseline_result.merge(
    gold_standard_result,on=["conversation_id"]
)
# full_result["prediction"] = full_result["prediction"].astype(bool).astype(int)
# full_result["gold_standard"] = full_result["gold_standard"].astype(bool).astype(int)

print(
    np.mean(full_result["prediction"] == full_result["gold_standard"])
)

0.625


## Crude test train split for judge training and validation

In [36]:
training_set = judge_dataset_examples[:110]
validation_set = judge_dataset_examples[110:]

## Run the optimization

In [44]:
judge_model = dspy.LM(
    "gemini/gemini-1.5-flash",
    api_key=secrets["GEMINI_API_KEY"],
    cache=False,
    temperature=0
)
dspy.configure(lm=judge_model,track_usage=True,adapter=dspy.JSONAdapter())
generate_judge_reasoning = dspy.ChainOfThought(SupportTranscriptJudge)

optimizer = dspy.MIPROv2(
    metric=match_judge_metric,
    auto="medium",
    init_temperature=1.0,
    seed=101
)

generate_judge_reasoning_optimized = optimizer.compile(
    generate_judge_reasoning,
    trainset=training_set,
    valset=validation_set,
    requires_permission_to_run=False,
)

2025/08/23 15:48:52 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 50

2025/08/23 15:48:52 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/08/23 15:48:52 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/08/23 15:48:52 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


  0%|                                                                                                                                                   | 0/110 [00:00<?, ?it/s]15:48:52 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:48:52 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:48:53 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


...

15:50:54 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


2025-08-23 15:50:54 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 29.00 / 49 (59.2%):  96%|██████████████████████████████████████████████████████████████████████████████████████████████████▉    | 48/50 [00:03<00:00, 14.53it/s]

15:50:54 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


Average Metric: 29.00 / 50 (58.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 14.52it/s]

2025/08/23 15:50:54 INFO dspy.evaluate.evaluate: Average Metric: 29 / 50 (58.0%)
2025/08/23 15:50:54 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 58.0 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 5'].
2025/08/23 15:50:54 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [66.0, 68.0, 62.0, 62.0, 68.0, 70.0, 70.0, 68.0, 62.0, 68.0, 70.0, 68.0, 74.0, 74.0, 72.0, 70.0, 72.0, 62.0, 58.0]
2025/08/23 15:50:54 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 74.0
2025/08/23 15:50:54 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/08/23 15:50:54 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 74.0!


In [45]:
optimized_score = evaluator(generate_judge_reasoning_optimized)

15:50:59 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:50:59 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:50:59 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:50:59 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:50:59 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


...

2025-08-23 15:51:03 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 109.00 / 159 (68.6%):  99%|█████████████████████████████████████████████████████████████████████████████████████████████████▊ | 158/160 [00:03<00:00, 43.48it/s]

15:51:03 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


2025-08-23 15:51:03 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 110.00 / 160 (68.8%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:03<00:00, 40.47it/s]

2025/08/23 15:51:03 INFO dspy.evaluate.evaluate: Average Metric: 110 / 160 (68.8%)


,transcript,example_satisfied,_id,reasoning,pred_satisfied,match_judge_metric
0,Company: American Airlines Transcript so far: Customer: I need thi...,True,example_0,Agent is trying to help but needs more information. The agent's r...,false,
1,Company: American Airlines Transcript so far: Customer: traveling ...,True,example_1,Agent repeatedly deflected the customer's request for assistance w...,false,
2,Company: American Airlines Transcript so far: Customer: Looks like...,False,example_2,The agent acknowledges the customer's statement but doesn't offer ...,false,✔️ [1]
3,Company: American Airlines Transcript so far: Customer: I am bring...,True,example_3,The agent's response is helpful and provides necessary information...,true,✔️ [1]
4,Company: American Airlines Transcript so far: Customer: I printed ...,True,example_4,Agent initially misunderstood but quickly clarified and provided t...,true,✔️ [1]
...,...,...,...,...,...,...
155,Company: American Airlines Conversation Transcript so far: Custome...,True,example_155,The agent apologized and offered further assistance with American ...,true,✔️ [1]
156,"Company: American Airlines Transcript so far: Customer: In email, ...",True,example_156,The agent acknowledged the problem and promised to forward it to t...,true,✔️ [1]
157,Company: American Airlines Transcript so far: Customer: I am looki...,False,example_157,The agent acknowledged the request and is actively working on it.,false,✔️ [1]
158,Company: Delta Air Lines Transcript so far: Customer: I fly from B...,False,example_158,The agent's response is not completely satisfactory as it does not...,false,✔️ [1]


In [46]:
optimized_score

68.75

## Check against validation set

In [47]:
evaluator_valid = dspy.Evaluate(
    metric=match_judge_metric,
    devset=validation_set,
    display_table=True,
    display_progress=True,
    num_threads=24,
)

In [48]:
optimized_valid_score = evaluator_valid(generate_judge_reasoning_optimized)

15:51:36 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:51:36 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:51:36 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


2025-08-23 15:51:36 - LiteLLM - INFO - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


15:51:36 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= gemini-1.5-flash; provider = gemini


...

2025-08-23 15:51:38 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 35.00 / 49 (71.4%):  96%|██████████████████████████████████████████████████████████████████████████████████████████████████▉    | 48/50 [00:01<00:00, 47.23it/s]

15:51:38 - LiteLLM:INFO: utils.py:1239 - Wrapper: Completed Call, calling success_handler


2025-08-23 15:51:38 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
Average Metric: 36.00 / 50 (72.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 29.97it/s]

2025/08/23 15:51:38 INFO dspy.evaluate.evaluate: Average Metric: 36 / 50 (72.0%)


,transcript,example_satisfied,_id,reasoning,pred_satisfied,match_judge_metric
0,Company: American Airlines Transcript so far: Customer: I started ...,True,example_110,Agent provided helpful information and alternative solutions.,true,✔️ [1]
1,Company: British Airways Transcript so far: Customer: We have a fl...,True,example_111,"The agent thoroughly addresses the customer's concerns, providing ...",true,✔️ [1]
2,Company: American Airlines Transcript so far: Customer: I have 2 s...,True,example_112,The agent initially gave a negative response but then offered a he...,true,✔️ [1]
3,Company: American Airlines Transcript so far: Customer: I want to ...,False,example_113,The agent did not answer the customer's questions about the AAdvan...,false,✔️ [1]
4,Company: American Airlines Transcript so far: Customer: We're due ...,True,example_114,Agent correctly identified the problem and gave a clear solution.,true,✔️ [1]
5,Company: American Airlines Transcript so far: Customer: I'm having...,True,example_115,The agent offered a helpful solution but didn't directly address t...,false,
6,Company: American Airlines Transcript so far: Customer: I'm travel...,False,example_116,The agent provided a helpful and accurate response by directing th...,true,
7,Company: American Airlines Transcript so far: Customer: Can I chec...,True,example_117,The agent provided a helpful and complete answer to the customer's...,true,✔️ [1]
8,Company: American Airlines Transcript so far: Customer: Could you ...,True,example_118,"The agent initially gave a correct, albeit unhelpful response, but...",true,✔️ [1]
9,Company: American Airlines Transcript so far: Customer: I work as ...,False,example_119,"The agent repeatedly fails to understand the customer's request, l...",false,✔️ [1]


In [49]:
optimized_valid_score

72.0

## Save the results

In [50]:
generate_judge_reasoning.save("dspy_modules/baseline_llm_judge",save_program=True)

In [51]:
generate_judge_reasoning_optimized.save("dspy_modules/optimized_llm_judge",save_program=True)

## Use this to see the resulting system prompt

In [52]:
generate_judge_reasoning_optimized.inspect_history(n=1)





[2025-08-23T15:51:38.410278]

System message:

Your input fields are:
1. `transcript` (str): Input transcript to judge
Your output fields are:
1. `reasoning` (str): 
2. `satisfied` (str): Whether the agent satisfied the customer query. This must be either True or False
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## transcript ## ]]
{transcript}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "satisfied": "{satisfied}"
}
In adhering to this structure, your objective is: 
        You are an expert in customer service, evaluating interactions between customers and agents. Analyze the following conversation transcript, considering the agent's response in context.  Assess whether the agent's response was satisfactory, focusing on whether it was helpful, accurate, polite, concise, and understanding.  Consider the customer's perspective.  Respond